In [4]:
import pandas as pd

# TSV-Datei einlesen
ab_ag_MMseqs2 = pd.read_csv('data/ab_ag_annotated.tsv', sep='\t')

# NUR die Spalte 'CDR_H1' extrahieren
CDR_H1_seq = ab_ag_MMseqs2['CDR_H1']

with open("CDR_H1_seqs.fasta", "w") as f:
    for i, seq in enumerate(CDR_H1_seq):
        # Wenn seq eine Liste ist, mach sie zu einem String
        if isinstance(seq, list):
            seq = ''.join(seq)
        elif isinstance(seq, str) and seq.startswith("[") and seq.endswith("]"):
            # Falls als String gespeicherte Liste: eval-sicher entschachteln
            seq = ''.join(eval(seq))
        f.write(f">seq{i}\n{seq}\n")

#Kontrolle (kann auch wieder entfernt werden)
print("Erste 3 FASTA-Sequenzen:")
with open("CDR_H1_seqs.fasta") as f:
    for _ in range(6):  # 3 Sequenzen = 6 Zeilen (je Header + Sequenz)
        print(f.readline().strip())

Erste 3 FASTA-Sequenzen:
>seq0
FTFSAYG
>seq1
FTFSAYG
>seq2
FTFSAYG


In [ ]:
#code im Terminal schreiben und die Bezüge, den Code dazu in hier eintragen, damit man weiß, was man macht
#die datei dann speichern und dann ist es auch okay
#COde nachdem Fasta datei erstellt wurde



#Erstellt eine interne MMseqs2-Datenbank mit Namen cdr_h1_db aus der FASTA-Datei CDR_H1_seqs.fasta.
mmseqs createdb CDR_H1_seqs.fasta cdr_h1_db


#Clustern
mmseqs cluster cdr_h1_db clu tmp --min-seq-id 0.9 -c 0.8 

#cdr_1_db: Ergebnis-Datenbank mit den Clustern
#tmp: temporärer Arbeitsornder
# --minseq-id 0.9: Mindestens 90% sequenz identität
# -c 0.8: Coverage: 80% der kürzeren Sequenz vom alignment müssen abgedeckt sein

#No k-mer could be extracted for the database tmp/7247275725010601612/input_step_redundancy.
#Maybe the sequences length is less than 14 residues. --> MMseqs2 unterstützt keinen cluster Befehl für seq < 14, auch runter reduzieren hilf nicht
mmseqs cluster cdr_h1_db clu tmp --min-seq-id 0.9 -c 0.8 --kmer-per-seq 5
mmseqs cluster cdr_h1_db clu tmp --min-seq-id 0.9 -c 0.8 --kmer-per-seq-scale 0.1
#haben beide nicht funktioniert
#padding vermutlich unnötig
